Topic: rGDP and compensation of employees

1. First of all, import libraries.

In [ ]:
import pandas as pd
import requests
import io
import os

2. Start to download data from internet.

In [ ]:
# Define URL for macroeconomic data
url_macro = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_rGDP.dta'
# Read Stata file into a pandas DataFrame
df_macro = pd.read_stata(url_macro)

# Define URL for OECD data
url_oecd = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN10@DF_TABLE1_INCOME,2.0/A.JPN........V..?dimensionAtObservation=AllDimensions&format=csv"
# Fetch data from the URL
response = requests.get(url_oecd)
response.raise_for_status()  # Raise an exception for HTTP errors
# Read the text data into an in-memory text buffer
data = io.StringIO(response.text)
# Read CSV data into a pandas DataFrame
df_oecd = pd.read_csv(data)

for the first line, set the URL for macroeconomic data, and then using pandas to read the web stata file, meanwhile save the things that we read from pandas.

for the second line, I collect the API from OECD database, and using OECD data explorer to filter the compensation of employees in the JPN, and also transforming the APIurl to csv form which can help us to read in pandas command. Then I used request to request the data from url, to avoid some internet error, the program will stop automatically when it occur some internet mistakes. And like before, use pandas to read and save the data.

3. Filter and Rename Columns

In [ ]:
# Filter macroeconomic data for Japan, select columns, and drop missing values
df_macro_jp = df_macro.query("ISO3 == 'JPN'")[['ISO3', 'year', 'UN_rGDP', 'WDI_rGDP']].dropna()
# Filter OECD data for Japan
df_oecd_jp = df_oecd[['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE', 'TRANSACTION', 'UNIT_MEASURE']].query("REF_AREA == 'JPN'")

# Rename columns in the Japan macroeconomic DataFrame
df_macro_jp = df_macro_jp.rename({"ISO3": 'country', "year": 'date'}, axis=1)

# Rename columns and drop unnecessary columns in the Japan OECD DataFrame
df_oecd_jp = df_oecd_jp.rename({"REF_AREA": 'country', "TIME_PERIOD": 'date', 'OBS_VALUE': 'COM'}, axis=1).drop(["TRANSACTION", "UNIT_MEASURE"], axis=1)

# Print debug information before index conversion
print("Dataset shapes:")
print(f"Macro data: {df_macro_jp.shape}")
print(f"OECD data: {df_oecd_jp.shape}")

print("\nMacro data sample:")
print(df_macro_jp.head())
print("\nOECD data sample:")
print(df_oecd_jp.head())

4. Index Types Match

In [ ]:
# Convert date column in macro data to integer type
df_macro_jp['date'] = df_macro_jp['date'].astype(int)

print("\nMacro data sample after type conversion:")
print(df_macro_jp.head())

because the date in macroeconomics database are come with float, I fail to merge many times because of this, in order to solve this, we are going to make it to int.

5. Set Index

In [ ]:
# Set 'country' and 'date' as the index for both DataFrames
df_macro_jp = df_macro_jp.set_index(['country', 'date'])
df_oecd_jp = df_oecd_jp.set_index(['country', 'date'])

print("\nData after setting indices:")
print("Macro data:")
print(df_macro_jp.head())
print("\nOECD data:")
print(df_oecd_jp.head())

Preparing for merging 2 database, so we set country and date as the index.

6. Merge DataFrames

In [ ]:
# Merge the two DataFrames based on the common index ('country', 'date') using an inner join
df_merge = pd.merge(
    df_macro_jp,
    df_oecd_jp,
    right_index=True,
    left_index=True,
    how='inner'
)

# Print merging result
print("\nMerge results:")
print(f"Merged data shape: {df_merge.shape}")
print(df_merge.head())

7. Export Data

In [ ]:
# Ensure the 'data/intermediate/' directory exists
os.makedirs("data/homework1/", exist_ok=True)
# Export the merged DataFrame to a CSV file in the 'data/intermediate/' directory
df_merge.to_csv("data/homework1/merged_data_jp.csv")

# Ensure the 'data/raw/' directory exists
os.makedirs("data/homework1/", exist_ok=True)
# Export the raw OECD DataFrame to a CSV file in the 'data/raw/' directory
df_oecd.to_csv("data/homework1/oecd_jp.csv")

print("\nFiles saved successfully!")

8. Data Preview and Summary

In [ ]:
# Make it looks fancy
print("\n" + "="*50)
print("DETAILED DATA PREVIEW AND SUMMARY")
print("="*50)


# Show merged data with original indices
print("\nComplete Merged Dataset:")
df_merge_display = df_merge.reset_index()
print(df_merge_display)